<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/Query_CTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WOS+SCI+SCP+PTJ+CTR QUERY

Queries for the bibliographic datasets for 
* Web of Science (WOS), 
* Scielo (SCI)
* Scopus  (SCP)
* Puntaje (UDEA)
* Center (CTR)
of the scientific articles of Universidad de Antioquia. 

They were created with:

[WOS_SCI_SCP_PTJ_CTR.ipynb](./WOS_SCI_SCP_PTJ_CTR.ipynb)

In [1]:
import os
VERSION='NEW'
if os.getcwd()=='/content':
    !pip install openpyxl xlrd wosplus > /dev/null

## functions

In [2]:
import pandas as pd
import wosplus as wp
pd.set_option('display.max_colwidth',200)
from venn import draw_venn, generate_colors
import numpy as np
import fuzzywuzzy.process as fwp
from fuzzywuzzy import fuzz

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [3]:
%%writefile drive.cfg
[FILES]
WOS_SCI_SCP_PTJ_CTR.json.gz=19E1C1kRk4I0V3uXojqko8-NEicWaPp1j
WOS_SCP_UDEA_SJR_SIU.xlsx=0BxoOXsn2EUNIQ3R4WDhvSzVLQ2s
Base_de_datos_investigadores_Definitiva.csv=12oalgUeKhpvzkTPBP8pXCeHTrF-KO223dy9ov9w9QKs
UDEA_authors_with_WOS_info.json=1o1eVT4JD0FMMICq_oxrTJOzWh47veBMw
produccion_fecha_vig_2003_2018.xlsx=1WbtX4K__TTLxXRjuLvqUYz9tuHCIlS5v
UDEA_WOS_SCI_SCP_PTJ.json=1OkVytKbxJwGvXZDkynkSoUDtkUOTaT4A

Overwriting drive.cfg


##  Load data bases

In [4]:
affil='Univ Antioquia'
drive_files=wp.wosplus('drive.cfg')

In [5]:
UDEAjsonfile='WOS_SCI_SCP_PTJ_CTR.json.gz'
UDEAjsonfile='UDEAtmp.json.gz'
tmp=drive_files.load_biblio(UDEAjsonfile,compression='gzip')# TODO CHANGE FOR LAST VERSION IN GOOGLE DRIVE
UDEA=drive_files.biblio['WOS'].copy().reset_index(drop=True)

In [6]:
from check_quality import *
check_quality(UDEA)

authors_WOS
13645
Leptonic charged Higgs decays in the Zee model ; authors_WOS: [] ; AU: Sierra, DA
Restrepo, D

Tipo contains UDEA
2240
UDEA_authors
10956
UDEA_authors → full_names (Extrapolado puntaje)
10956
UDEA_authors → "NOMBRE COMPLETO" (Extrapolado CENTRO)
7329
TI: "The inert doublet model" check WOS_author vs UDEA_authors
...


## Indices

See: 

`{'CÉDULA': 63508258.0,
  'DEPARTAMENTO': 'Instituto de Biología',
  'FACULTAD': 'Facultad de Ciencias Exactas y Naturales',
  'GRUPO': 'Sin Grupo Asociado',
  'INICIALES': 'I.',
  'NOMBRE COMPLETO': 'Idalyd Fonseca Gonzalez',
  'NOMBRES': 'Idalyd',
  'PRIMER APELLIDO': 'Fonseca',
  'SEGUNDO APELLIDO': 'Gonzalez',
  'WOS_affiliation': ['Univ Antioquia, Colombia.'],
  'WOS_author': ['FONSECA, IDALYD',
   'FONSECA-GONZALEZ, IDALYD',
   'Fonseca-Gonzalez, Idalyd',
   'Fonseca-Gonzalez, I.'],
  'full_name': 'FONSECA GONZALEZ IDALYD'}`

In [7]:
#TODO: Improve indices
def get_groups(l,g):
    for d in l:
        gt=d.get('GRUPO')
        if gt and type( gt )==str:
            gs=gt.replace(
                ', Grupo','; Grupo'
            ).split('; ')
            for gg in gs:
                if gg not in g:
                    g.append(gg)
    return g


json_column='UDEA_authors'
facultades={'key':'FACULTAD',
            'values' : UDEA.UDEA_authors.apply(lambda l: 
                         [d.get('FACULTAD') for d in l] 
                          if type(l)==list else None
                          ).dropna().apply(pd.Series).stack().unique()}
departamentos={'key':'DEPARTAMENTO',
            'values' :UDEA.UDEA_authors.apply(lambda l: 
                         [d.get('DEPARTAMENTO') for d in l] 
                          if type(l)==list else None
                        ).dropna().apply(pd.Series).stack().unique()}
g=[]
#append to g
tmp=UDEA.UDEA_authors.apply(lambda l: 
                        get_groups(l,g)
        if type(l)==list else None
                        )
grupos={'key':'GRUPO',
            'values' :g}
nombre_completo={'key'    : 'NOMBRE COMPLETO',
                 'values' : UDEA[json_column].apply(lambda l: 
                            [d.get('NOMBRE COMPLETO') for d in l] 
                            if type(l)==list else None
                            ).dropna().apply(pd.Series).stack().unique()
                }#Nombres Apellidos. #TODO: Be sure to include full list
full_name={'key'    : 'full_name',
                 'values' : UDEA[json_column].apply(lambda l: 
                            [d.get('full_name') for d in l] 
                            if type(l)==list else None
                            ).dropna().apply(pd.Series).stack().unique()
                }#Apellidos Nombres . #TODO: Be sure to include full list

## Query function

In [9]:
def query_json_column(q,df=UDEA,json_column='UDEA_authors',
                        choices=nombre_completo,scorer=fuzz.partial_token_sort_ratio,**kwargs):
    #Found best exact match from index
    fchoices=fwp.extractOne(q,choices['values'],scorer=scorer)[0]
    # Exact search in indexed subcolumn converted to strins (e.g list → string if necessary)
    dfF=df[df[json_column].apply(lambda l: True in [ str(d.get(choices['key'])).find(fchoices)>-1 
                                    for d in l if d.get(choices['key'])] if type(l)==list else False)]
    return dfF.reset_index(drop=True)

In [10]:
r=query_json_column('Diego Alejandro Restrepo Quintero',df=UDEA,json_column='UDEA_authors',
                        choices=nombre_completo,scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)

In [11]:
r.shape

(36, 181)

In [13]:
r[['TI','AU','authors_WOS',json_column]].reset_index(drop=True)[:2]

,TI,AU,authors_WOS,UDEA_authors
0,The inert doublet model,"Arias, C\nMartins, J\nMartinez, H\nRon, E\nSalzmann, C\nVasconcelos, GMS\nVillalba, F\n","[{'WOS_author': 'Arias, C.', 'affiliation': ['Univ Antioquia, Inst Fis, AA 1226, Medellin, Colombia.'], 'i': 0}]","[{'SEGUNDO APELLIDO': 'Noreña', 'NOMBRE COMPLETO': 'Oscar Alberto Zapata Noreña', 'full_name': 'ZAPATA NOREÑA OSCAR ALBERTO', 'INICIALES': 'O. A.', 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': ..."
1,Leptonic charged Higgs decays in the Zee model,"Sierra, DA\nRestrepo, D\n",[],"[{'SEGUNDO APELLIDO': 'Quintero', 'NOMBRE COMPLETO': 'Diego Alejandro Restrepo Quintero', 'full_name': 'RESTREPO QUINTERO DIEGO ALEJANDRO', 'INICIALES': 'D. A.', 'DEPARTAMENTO': 'Instituto de Físi..."


In [14]:
r=query_json_column('Instituto de Física',df=UDEA,json_column='UDEA_authors',
                        choices=departamentos,scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)

In [15]:
r.shape

(849, 181)

In [16]:
r=query_json_column('Grupo de Fenomenología de Interacciones Fundamentales',df=UDEA,json_column='UDEA_authors',
                        choices=grupos,scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)

In [17]:
r.shape

(80, 181)

In [18]:
gdf=pd.DataFrame()
for g in grupos['values']:
    r=query_json_column(g,df=UDEA,json_column='UDEA_authors',choices=grupos,
                        scorer=fuzz.partial_token_sort_ratio,score_cutoff=79)
                        
    gdf=gdf.append( {'Group':g,'articles':r.shape[0]},ignore_index=True )
gdf['articles']=gdf['articles'].astype(int)

In [19]:
gdf.sort_values('articles',ascending=False).reset_index(drop=True)[:10]

,Group,articles
0,Sin Grupo Asociado,399
1,Grupo de Materia Condensada-UdeA,299
2,Grupo de Estado Sólido,269
3,"Grupo Reproducción, Inmunovirología, Infección y Cáncer",265
4,Inmunovirología,265
5,"Centro de Investigación, Innovación y Desarrollo de Materiales - CIDEMAT - Anteriormente: Grupo de Corrosión y Protección",251
6,"Centro de Investigación, Innovación y Desarrollo de Materiales - CIDEMAT - Anteriormente: Grupo de Corrosión y Protección,",251
7,"Grupo de Neurociencias de Antioquia, SINAPSIS",227
8,Grupo de Neurociencias de Antioquia,227
9,Grupo Académico de Epidemiología Clínica,224
